# Deriving a measure of neighborhood connectedness
1. Download the OSM data for the city using OSMnx library in Python.


In [10]:
import osmnx as ox
import pandas as pd 
import geopandas as gpd
import networkx as nx
import shapely

In [3]:
# read neighborhoods
koeln = gpd.read_file('../data/raw/3 Neighborhoods/Neighborhoods_Köln.gpkg')
koeln = koeln.to_crs(epsg = 4326)

#  create neighborhoods centroids
koeln['centroid'] = koeln.centroid

/var/folders/gp/knk8fb1937d_5p9ybs31txzr0000gn/T/ipykernel_23395/1224768527.py:6: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  koeln['centroid'] = koeln.centroid


In [6]:
## Extract the stops 
# download the "koeln" city boundary polygon
city = ox.geocode_to_gdf('Koeln, Germany')
city_polygon = city.loc[0, 'geometry']

# get public transport stops within the city boundary polygon
tags = {'bus': True, 'tram': True, 'train': True}
stops = ox.geometries_from_polygon(polygon=city_polygon, tags=tags)

# Koeln netowrk 
G = ox.graph_from_polygon(city_polygon, network_type='walk', simplify=False)
graph_nodes, graph_edges = ox.graph_to_gdfs(G)

/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shapely/set_operations.py:340: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)
/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shapely/constructive.py:181: RuntimeWarning: invalid value encountered in buffer
  return lib.buffer(
/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/homebrew/Caskroom/miniconda/base/envs/ifohack_spatial_py310_small_v1/lib/python3.10/site-packages/shapely/set_operatio

In [7]:
# Nearest nodes to centroids 
koeln['nb_start'] = ox.nearest_nodes(G, koeln.centroid.x, koeln.centroid.y)

/var/folders/gp/knk8fb1937d_5p9ybs31txzr0000gn/T/ipykernel_23395/2963752950.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  koeln['nb_start'] = ox.nearest_nodes(G, koeln.centroid.x, koeln.centroid.y)
/var/folders/gp/knk8fb1937d_5p9ybs31txzr0000gn/T/ipykernel_23395/2963752950.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  koeln['nb_start'] = ox.nearest_nodes(G, koeln.centroid.x, koeln.centroid.y)


In [51]:
# Filter out Polygon geometries and keep only Point geometries
points = stops.loc[stops.geometry.geom_type == 'Point']

# Select only the rows where element_types is equal to "node"
points = points.loc[points.index.get_level_values('element_type') == 'node']

# Reset the index
points = points.reset_index(drop=True)

In [52]:
points

,bus,highway,name,public_transport,geometry,VRS:gemeinde,VRS:name,VRS:ortsteil,VRS:ref,railway,...,sidewalk:left:bicycle,building:part,disused:name,disused:public_transport,disused:railway,indoor,caravans,bus_routes,ways,type
0,yes,bus_stop,CityTour Haltestelle 7: Schokoladenmuseum Rhei...,stop_position,POINT (6.96278 50.93187),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,yes,NaN,Wiso-Fakultät,stop_position,POINT (6.92948 50.93110),KÖLN,Wiso-Fakultät,Innenstadt,11652,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,yes,NaN,Bonner Straße/Gürtel,stop_position,POINT (6.96564 50.90371),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,yes,NaN,Leiblplatz,stop_position,POINT (6.91553 50.92565),KÖLN,NaN,Lindenthal,13053,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Eifelplatz,stop_position,POINT (6.94351 50.92335),KÖLN,NaN,Innenstadt,11509,tram_stop,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2465,yes,bus_stop,Bürgeramt Rodenkichen,platform,POINT (6.99876 50.87261),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2466,yes,bus_stop,Bugenhagenstraße,platform,POINT (7.02417 50.98646),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2467,yes,bus_stop,Am Flachsrosterweg,platform,POINT (7.02214 50.98339),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2468,yes,bus_stop,Heimfriedweg,platform,POINT (7.03147 50.98401),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Loop over neighborhoods and stops 
routes_list = []
for nb in koeln.Neighborhood_FID:
    for idx, stop in points.iterrows():
        dest_id = ox.nearest_nodes(G, points.iloc[idx].geometry.x, points.iloc[idx].geometry.y)
        route = nx.shortest_path(G, koeln.loc[koeln.Neighborhood_FID == nb].nb_start, dest_id, weight = "length")
        route_points = graph_nodes.loc[route]
        route_line = shapely.LineString(list(route_points.geometry.values))
        routes_list.append(route_line)

NodeNotFound: Either source 0    1892290483
Name: nb_start, dtype: int64 or target 309352 is not in G

2. Extract the street network from the OSM data using OSMnx.
3. Compute the accessibility to public transport by identifying the locations of public transport stops such as bus stops and train stations, and computing the distance from each neighborhood to the nearest public transport stop using the network distance along the street network.
4. Compute the accessibility to other neighborhoods by computing the shortest path between each pair of neighborhoods using the street network and calculating the average distance between each neighborhood and all other neighborhoods.
5. Compute the commuting distance to the commercial center by identifying the location of the commercial center and computing the shortest path from each neighborhood to the commercial center using the street network.
6. Compute the commuting distance to working neighborhoods by identifying the locations of working neighborhoods and computing the shortest path from each neighborhood to the nearest working neighborhood using the street network.
7. Combine these measures into a single measure of connectedness using a weighted sum of the individual measures, where the weights are chosen based on the relative importance of each measure.

In [ ]:
koeln['centroid'] = koeln.centroid
route = nx.shortest_path(koeln.centroid.iloc[0], )

In [ ]:
# compute the distance from each neighborhood centroid to the nearest public transport stop
koeln['pt_distance'] = koeln.centroid.apply(lambda x: ox.distance.nearest_nodes(city_polygon, stops.longitude, stops.latitude)[1])
koeln['pt_distance'] = koeln['pt_distance'].apply(lambda x: ox.distance.euclidean_dist_vec(*x))